# 0 摘要

Deep Walk是一种学习网络图中结点表达的新方法。它通过学习网络中结点之间的关系，将图中结点表征为低维连续向量。

在具体方法上采用截断随机游走(Truncated random walks)方式来生成样本序列，从而使用网络局部信息来进行学习。

# 1 简介

Deep Walk借鉴了NLP中无监督学习的方式，通过随机游走构造结点序列，从而进行无监督学习算法。Deep Walk的主要贡献在于：

- 介绍了一种对图的统计学习建模方法，能够通过短随机游走学习图中结构规律；
- 相比于其它算法都有很大提升；
- 算法可进行大规模学习与扩展。

# 2 问题定义

定义图$G=(V, E)$，其中$V$是网络中的结点集合，$E$是网络中的边集合。给定一个部分标注的社交网络图$G_L=(V, E, X, Y)$，其中：

- $X\in \mathbb{R}^{|V|\times S}$，其中$S$是样本特征空间维度
- $Y\in \mathbb{R}^{|V|\times |\mathbb{\gamma}|}$，其中$\gamma$是目标集合

我们的目的是通过网络学习一个假设$\mathcal{H}$将每一个样本$X$映射到$Y$

传统机器学习分类：
- 关系分类（relational classification）
- 将这一问题定义为Undirected Markov Network推断问题，从而计算后验概率

Deep Walk：
- 无监督学习
- 利用结点之间的关系信息
- 将图中结构信息空间与目标变量空间解耦
- 低维稠密表示可以用作下游任务

# 3 Learning Social Representations

Social Representations应该具备以下性质：

- Adaptability：网络局部结构更新很快，新节点加入就会发生局部关系变化；因此要避免重复读取整张图进行学习，而是进行自适应学习新的网络关系；
- Community Aware：任意结点间的向量距离应该能够衡量网络中结点的相似性；
- Low Dimensional：当有标注数据非常稀少时，低维模型的泛化性能更好，收敛速度更快；
- Continuous：向量表征要在连续空间中对局部网络社区中的关系信息进行建模，另外Continuous representation在分类任务中能够有更加平滑的分类边界。

## 3.1 Random Walks

我们定义：

- 从结点$v_i$开始的随机游走路径为$\mathcal{W}_{v_i}$
- $\mathcal{W}_{v_i}$代表的是一个由$\mathcal{W}_{v_i}^1, \mathcal{W}_{v_i}^2, \cdots$构成的随机过程。例如$\mathcal{W}_{v_i}^{k+1}$就是在以结点$v_i$为root的随机过程中，结点$v_k$的邻居

> 随机游走被用作内容推荐问题和社区发现问题中的一种相似度衡量方式。另外，在output sensitive algorithms这一类算法中用作计算local community structure information。

使用随机游走来进行local community information学习有以下优点：

- 工程上可以实现局部多条路线并行化游走，例如多个random walkers可以同时遍历整张图的不同部分；
- 随机游走只依赖于短路径的遍历，因此可以更好地适应网络中局部微小关系的变动，而不需要遍历整张图来学习变化

## 3.2 Connection: Power Laws

如果在一个联通图中，结点的度（degree）的分布符合幂定律，那么它就是无标度网络（scale-free），我们可以观察到无标度网络遵循“重尾”的幂分布，作者分别使用了两张图来表达：

![](power-law.png)

在language modeling中考虑了语料中词频服从幂分布，Deep Walk的贡献就是将NLP中的建模假设前提重新用在了community structure建模中。

> 除了scale-free网络（无标度网络）外，还有随机网络，随机网络是指网络中结点的degree相对均匀，不存在长尾现象。例如，像微博用户的社交关系分布就符合无标度网络。

## 3.3 Language Modeling

语言模型：

- 给定一个词的序列$W_1^n = (w_0, w_1, \cdots, w_n)$
- 最大化似然概率$P(w_n|w_0, w_1,\cdots,w_{n-1})$

图模型：

- 给定一个short random walk序列$V_1^n=(v_0, v_1, \cdots, v_n)$
- 最大化似然概率$P(v_i|v_0, v_1, \cdots, v_{i-1})$

参考Word2Vec算法，学习结点的co-occurrences只是fake task，真正想要的是结点的representations，因此需要一个mapping函数$\Theta$，将$v\in V$映射到$\mathbb{R}^{|V|\times d}$的向量空间，才可以进行后续的计算。

在Neural Network语言模型中，例如word2vec，在算法中做了如下优化：

- 用中心词预测左右的上下文；
- 上下文包含了出现在中心词左侧和右侧的词；
- 忽略上下文中词顺序信息（即不考虑上下文词与中心词的距离）

即最终优化问题可以写作：

$$\min -\log P(\{v_{i-w}, \cdots, v_{i-1}, v_{i+1}, \cdots, v_{i+w}\}|\Theta(v_i))$$

这些词的优化方式对于social representation learning也是至关重要的：

- 顺序无关性假设能够更好地capture随机游走中“nearness“的概念
- 同时也可以帮助加速building small models的训练时间

最终基于上述的内容，Deep Walk可以学到低维稠密向量表示，并且具有相似邻居的结点，其在向量空间中的表征也更加接近。

# 4 Deep Walk算法

## 4.1 介绍

Deep Walk算法包含两部分：
- random walker generator:
    - 随机从输入图G中均匀采样结点$v_i$作为随即游走的根节点
    - 以$v_i$为根节点开始随机游走，直到序列长度达到$t$（作者提到$t$可以是变长的）
    - 以$v_i$为根节点的多条路径上随机游走，产生$\gamma$条序列
- update procedure
    - 采用skip-gram进行模型参数更新
    - Huffman Tree构建二叉树，进行优化，时间复杂度由O(V)降至O(logV)

我们定义模型的random walker生成部分：

![](deep-walk.png)

模型更新部分：

![](update.png)

## 4.2 模型变体

### 4.2.1 Streaming

流式学习：无需知道整体graph的前提下进行小步学习。需要使用较低学习率+长时间优化收敛；无法直接根据结点的degree构建huffman tree，因此需要对结点degree有一定的先验；

### 4.2.2 Non-random walks

非随机游走。有些序列本身是用户与